<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js023_NaClIonMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js023_NaClIonMD2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.09.20 created,  last updated on 2024.04.30
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js023_NaClIonMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js023_NaClIonMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.08 created, last updated on 2018.11.16
//    ver 0.0.1  2019.01.14 v1, last updated on 2021.05.12
//    ver 0.0.2  2021.10.30 v2, last updated on 2021.10.30
//    ver 0.0.3  2023.03.09 v3, last updated on 2023.08.20
//
// --------------------  molecular dynamics 2D
//
//   method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//   force: ion f(r) = fc + fr + fa,  fc:Debye-screened Coulomb force
//      fc = eForceConst*zi*zj*(EXP(-r/6.5e-10)/r)*(1.0/r+1.0/6.5e-10)
//      fr = 6.9742e-11*EXP((a-r)/b)    :repulsive force
//      fa = -6.9742e-21*(c/r^6)        :attractive force
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), 'near' means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const NaClIonMD2D = (function(){ // ====================  NaClIonMD2D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = EE(J)
	const g_eForceConst = 1.0/(4.0*Math.PI*8.85418782e-12); // epsilon0=8.85418782e-12
	const g_nMax = 2000;						// array max

	let g_ionKind1 = 3;							// 3:Na+
	let g_ionKind2 = 7;							// 7:Cl-
	let g_nParticles = 100;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep =  2.0*1.0e-15;				// (s) time step dt
	let g_xMax = 6.0E-9;						// (m) x-Box size
	let g_yMax = 6.0E-9;						// (m) y-Box size
	let g_rCutoff = 1.0e-9;						// force cutoff length
	let g_hh = 1.0e-12;							// forceTable r-division
	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) total potential energy

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle 0:He 1:Ne 2:Ar 3:Kr 4:Xe 5:Hg
	const g_potentialTable = dim3( 18, 18, 1002 );	// potential table [V[0], V[hh], V[2hh],..., V[rCutoff] ]
	const g_forceTable = dim3( 18, 18, 1002 );	// force table [F[0], F[hh], F[2hh],..., F[rCutoff] ]
	const g_reg = dimInt2( g_nMax, 100 );		// reg[i][0]:total number of particles near i-th particle
												// reg[i][j]:particle number near i-th particle, (j>0)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  SXion potential data  --------------------

	//    0 mass,1 charge,     2  a ,     3  b , 4 c , 5  r-ion,  6  color,  7 str
	const g_SXion = [
		[ 10.81*g_AMU,  3.0*g_EE, 0.720e-10, 0.080e-10,  0.0, 0.23e-10, "#ff4060", "B+++"  ], //  0
		[ 16.00*g_AMU, -2.0*g_EE, 1.626e-10, 0.085e-10, 20.0, 1.40e-10, "#4000ff", "O--"   ], //  1
		[ 19.00*g_AMU, -1.0*g_EE, 1.565e-10, 0.085e-10, 20.0, 1.33e-10, "#0040ff", "F-"    ], //  2
		[ 22.99*g_AMU,  1.0*g_EE, 1.260e-10, 0.080e-10, 20.0, 1.02e-10, "#ff0000", "Na+"   ], //  3
		[ 24.31*g_AMU,  2.0*g_EE, 1.161e-10, 0.080e-10, 10.0, 0.72e-10, "#ff6000", "Mg++"  ], //  4
		[ 26.98*g_AMU,  3.0*g_EE, 1.064e-10, 0.080e-10,  2.0, 0.53e-10, "#ff4080", "Al+++" ], //  5
		[ 28.09*g_AMU,  4.0*g_EE, 1.012e-10, 0.080e-10,  0.0, 0.40e-10, "#ff8040", "Si++++"], //  6
		[ 35.45*g_AMU, -1.0*g_EE, 1.950e-10, 0.090e-10, 30.0, 1.81e-10, "#0000ff", "Cl-"   ], //  7
		[ 39.10*g_AMU,  1.0*g_EE, 1.595e-10, 0.080e-10, 15.0, 1.38e-10, "#ff4000", "K+"    ], //  8
		[ 40.08*g_AMU,  2.0*g_EE, 1.414e-10, 0.080e-10, 10.0, 1.00e-10, "#ff8000", "Ca++"  ], //  9
		[ 47.88*g_AMU,  4.0*g_EE, 1.235e-10, 0.080e-10,  0.0, 0.61e-10, "#ff8060", "Ti++++"], // 10
		[ 87.62*g_AMU,  2.0*g_EE, 1.632e-10, 0.080e-10, 15.0, 1.16e-10, "#ffaf00", "Sr++"  ], // 11
		[ 137.3*g_AMU,  2.0*g_EE, 1.820e-10, 0.080e-10, 20.0, 1.36e-10, "#ffbf00", "Ba++"  ], // 12
		[ 4.003*g_AMU,  0.0*g_EE, 1.200e-10, 0.110e-10, 4.76, 1.28e-10, "#00ff80", "He"    ], // 13
		[ 20.18*g_AMU,  0.0*g_EE, 1.415e-10, 0.112e-10,11.03, 1.37e-10, "#00ff40", "Ne"    ], // 14
		[ 39.95*g_AMU,  0.0*g_EE, 1.878e-10, 0.117e-10,38.53, 1.70e-10, "#00ff00", "Ar"    ], // 15
		[ 83.80*g_AMU,  0.0*g_EE, 2.041e-10, 0.130e-10,55.33, 1.83e-10, "#40ff00", "Kr"    ], // 16
		[ 131.3*g_AMU,  0.0*g_EE, 2.258e-10, 0.145e-10,85.55, 1.99e-10, "#80ff00", "Xe"    ] ];//17

	const g_rCollision = [];		// (m) V(r) = 0 at r==g_rCollision[kind]
	const g_rBond = [];			// (m) bpnd length == 2.0*g_rBond[kind]
	const g_massOf = [];			// (kg) massOf[kind] : mass of kind
	const g_ionStrOf = [];		// strOf[kind] : string of kind, such as "Na+"
	const g_colorOf = [];			// colorOf[kind] : color of kind

	(function() {
		const n=g_SXion.length;
		for (let i=0; i<n; i++) {
			g_rCollision[i] = g_SXion[i][5]/1.12246;
			g_rBond[i] = g_SXion[i][5];
			g_massOf[i] = g_SXion[i][0];
			g_ionStrOf[i] = g_SXion[i][7];
			g_colorOf[i] = g_SXion[i][6];
		}
	}());

	//					0      1      2     3      4      5      6      7      8      9
	const g_strOf = [  "B",  "O",  "F", "Na", "Mg", "Al", "Si", "Cl",  "K", "Ca",
						"Ti", "Sr", "Ba", "He", "Ne", "Ar", "Kr", "Xe" ];

	function colorStrOf(kind) {
		return "<span style='color:"+g_colorOf[kind]+"'>"+g_strOf[kind]+"</span>";
	}

	//--- set forceTable[kindi][kindj][ir] and potentialTable[kindi][kindj][ir]

	function setForceTable() {
		const hh=g_hh;
		for (let ki=0; ki<18; ki++) {
			for (let kj=0; kj<18; kj++) {
				const a = g_SXion[ki][2]+g_SXion[kj][2];
				const b = g_SXion[ki][3]+g_SXion[kj][3];
				const c = g_SXion[ki][4]*g_SXion[kj][4]*1.0e-60;
				const zi = g_SXion[ki][1];
				const zj = g_SXion[kj][1];
				for (let ir=1; ir<=1001; ir++) {
					const r = ir*hh;
					const fc = g_eForceConst*zi*zj*(Math.exp(-r/6.5e-10)/r)*(1.0/r+1.0/6.5e-10);
					const fr = 6.9742e-11*Math.exp((a-r)/b); //repulsive force;
					const fa = -6.9742e-21*(c/(r*r*r*r*r*r)); //attractive force;
					g_forceTable[ki][kj][ir] = cutoff(r)*(fc + fr + fa);
				}
				g_forceTable[ki][kj][0] = g_forceTable[ki][kj][1] + g_forceTable[ki][kj][2];
				g_potentialTable[ki][kj][1001] = 0.0;
				for (let ir=1000; ir>=0; ir--) {
					g_potentialTable[ki][kj][ir] = g_potentialTable[ki][kj][ir+1]+g_forceTable[ki][kj][ir]*hh;
				}
			}
		}
	}

	function cutoff(r) {
		let ret;
		if (r>0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		} else {
			ret = 0.0;
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( material, boxSizeInNM, contTemp ) {
		const n=6;
		let lattice;
		g_sysTime = 0.0;
		setForceTable();
		g_xMax = boxSizeInNM*1.0e-9;
		g_yMax = boxSizeInNM*1.0e-9;
		if (material==0) { // NaCl
			g_ionKind1 = 3; // Na+
			g_ionKind2 = 7; // Cl-
			lattice = 5.6407e-10*0.99; //0.99: correction factor
		} else if (material==1) { // MgO
			g_ionKind1 = 4; // Mg++
			g_ionKind2 = 1; // O--
			lattice = 4.212e-10*0.94; //0.94: correction factor
		} else if (material==2) { // CaO
			g_ionKind1 = 9; // Ca++
			g_ionKind2 = 1; // O--
			lattice = 4.80e-10*0.94; //0.94: correction factor
		} else if (material==3) { // BaO
			g_ionKind1 = 12; // Ba++
			g_ionKind2 = 1;  // O--
			lattice = 5.536e-10*1.00;
		} else if (material==4) { // NaF
			g_ionKind1 = 3;  // Na+
			g_ionKind2 = 2;  // F-
			lattice = 4.62e-10*0.99;
		} else if (material==5) { // KF
			g_ionKind1 = 8;  // K+
			g_ionKind2 = 2;  // F-
			lattice = 5.34e-10*1.00;
		} else if (material==6) { // KCl
			g_ionKind1 = 8;  // K+
			g_ionKind2 = 7;  // Cl-
			lattice = 6.29e-10*0.99;
		}
		const s = 0.5*(g_xMax - lattice*n);
		g_nParticles = setNaClTypeBlock(0, g_ionKind1, g_ionKind2, n, n, lattice, s, s);
		ajustVelocity(contTemp);
	}

	function setNaClTypeBlock(ii, knd1, knd2, nx, ny, lattice, xPos, yPos) {
		let ipp = ii;
		const a = lattice/2.0;
		for (let i=0; i<2*nx; i++) {
			for (let j=0; j<2*ny; j++) {
				let knd;
				const x = xPos + a*i;
				const y = yPos + a*j;
				if ((i+j)%2==0) {
					knd = knd1;
				} else {
					knd = knd2;
				}
				setParticle(ipp, knd, x, y)
				ipp = ipp + 1;
			}
		}
		return ipp;
	}

	function setParticle(i, knd, x, y) {
		g_xx[i] = x;
		g_yy[i] = y;
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
		g_kind[i] = knd;
		g_mas[i] = g_massOf[knd];
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {
		if (tempMode==1) ajustVelocity(contTemp);
		registerNearParticles();
		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i]
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10;
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0;
		}

		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r,kndi,kndj) { // forceTable - linear interporation
		const hh=g_hh;
		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[kndi][kndj][ir] + a*g_potentialTable[kndi][kndj][ir+1])/hh;
		return ((hh-a)*g_forceTable[kndi][kndj][ir] + a*g_forceTable[kndi][kndj][ir+1])/hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- registeration

	function registerNearParticles() {
		const nn=g_nParticles;
		const rCut = g_rCutoff+20*2000*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;
		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek=0.0;  //kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( material, boxSizeInNM, contTemp)
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp )

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() { return [ g_ionKind1, g_ionKind2, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getKindStr:		function(kind) { return g_strOf[kind]; },

		getBondLength:	function(i,j) { return ( g_rBond[g_kind[i]] + g_rBond[g_kind[j]] ); },
		getChargeProduct: function(i,j) { return g_SXion[g_kind[i]][1]*g_SXion[g_kind[j]][1]; },
		getNearList:	function(i) { return g_reg[i]; },
		getIonData:		function(i) { return [ g_kind[i], g_rCollision[g_kind[i]], g_rBond[g_kind[i]] ]; },

		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  NaClIonMD2D end  ====================


const js023 = (function(){ // ====================  js Module  ====================

	const theModule = NaClIonMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 0;			// 0:NaCl  1:MgO  2:CaO  3:BaO  4:NaF  5:KF  6:KCl
	let v_BoxSizeInNM = 5.0;
	let v_contTemp = 300.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control

	let p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;
	let nowData = [];
  let kindList = [];
  let xxList = [];
	let yyList = [];
	let vxList = [];
	let vyList = [];

	let nCalc = 1;
	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

	let breakFlag = false;
	let perticleFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_BoxSizeInNM, v_contTemp );
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode, v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

	function setParticlesData() {
		nowData = [ sysTime, temperature, kineticEnergy, potentialEnergy ];
		kindList = [];
		xxList = [];
		yyList = [];
		vxList = [];
		vyList = [];
    for (let i=0; i<p_nParticles; i++) {
			let x, y, vx, vy;
      kindList[i] = theModule.getIonData(i)[0];
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}


	// --------------------  draw  --------------------

	const ballColor = [
		/* 0:B3+  */ "#ff4060", /* 1:O--  */ "#8000ff", /* 2:F-   */ "#0080ff", /* 3:Na+  */ "#ff0000",
		/* 4:Mg++ */ "#ff6000", /* 5:Al4+ */ "#ff4080", /* 6:Si4+ */ "#ff8040", /* 7:Cl-  */ "#4444ff",
		/* 8:K+   */ "#ff4000", /* 9:Ca++ */ "#ff8000", /*10:Ti4+ */ "#ff8060", /*11:Sr++ */ "#ffaf00",
		/*12:Ba++ */ "#ffbf00" ];

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 5, xSize = 400, ySize = 400, scale = xSize/p_xMax, yTextPos= 430;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode==0 ) {
			drawBalls( ctx, scale, xp, yp );
		} else if ( dispMode==1 || dispMode==2 ) {
			strokeBalls( ctx, scale, xp, yp );
			drawBonds( ctx, dispMode-1, scale, xp, yp );
		} else if ( dispMode==3 ) {
			drawBonds( ctx, 2, scale, xp, yp );
			drawBalls( ctx, scale, xp, yp, 0.4 );
		}

		// caption
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();

		ctx.fillStyle = ballColor[p_kind1];
		ctx.fillText(theModule.getKindStr(p_kind1), xp, yTextPos+20);
		if ( p_kind1 != p_kind2 ) {
			ctx.fillStyle = ballColor[p_kind2];
			ctx.fillText(theModule.getKindStr(p_kind2), xp+30, yTextPos+20);
		}
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos);
		ctx.fillText(`,   N = ${p_nParticles}`, xp+60, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)} x ${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, scale, xp, yp, ballScale ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		const bScale = (ballScale==undefined) ? 1.0 : ballScale;
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getIonData(i);
			ctx.fillStyle = ballColor[kind];
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rCollision*scale*bScale, 0, twoPi, false);
			ctx.fill();
		}
	}

	function strokeBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getIonData(i);
			ctx.strokeStyle = "#444444";
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rBond*scale, 0, twoPi, false);
			ctx.stroke();
		}
	}

	function drawBonds( ctx, bondMode, scale, xp, yp ) { // single kind only - using nearList
		const nn = p_nParticles, yMax = p_yMax, pi = Math.PI;
		for (let i=0; i<nn-1; i++) {
			let xi, yi; [ xi, yi ] = theModule.getPosition(i);
			const nearList = theModule.getNearList(i);
			for (let k=1; k<nearList[0]; k++) {
				const j = nearList[k];
				if ( theModule.getChargeProduct(i,j)>0 ) continue;
				let xj, yj; [ xj, yj ] = theModule.getPosition(j);
				const xij = xi-xj, yij = yi-yj;
				const dij = Math.sqrt(xij*xij+yij*yij);
				const d0 = theModule.getBondLength(i,j);
				if ( dij<1.2*d0 ) {
					let hue;
					if ( bondMode==0 ) { // bond color -- length
						hue = Math.floor((dij/d0-1.0)*900+120);
					} else if ( bondMode==1 ) { // bond color -- direction
						const th = 2.0*(Math.atan2(yij,xij)+0.5*pi)/pi;
						//const th = 3.0*(Math.atan2(yij,xij)+0.5*pi)/pi;
						hue = Math.floor((th-Math.floor(th))*360.0);
					}
					if (hue<0) hue = 0;
					if (hue>270) hue = 270;
					const color = (bondMode==2) ? `#444444` : `hsl(${hue},100%,50%)`;
					drawLine(ctx, xi*scale+xp, (yMax-yi)*scale+yp, xj*scale+xp, (yMax-yj)*scale+yp, color );
				}
			}
		}
	}

	function drawLine(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

	function breakLoop() {
		breakFlag = true;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, kindList, xxList, yyList, vxList, vyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
		pygetParticlesList, pygetParticlesList, //() :return [ nowData, kindList, xxList, yyList, vxList, vyList ]
	};

})(); // ====================  js023 module end  ====================


const js = js023;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js023] ionic crystal - ion Molecular Dynamics MD2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme: </label>
<select id="slct_theme" onChange="js.setTheme()">
<option>0:NaCl</option><option>1:MgO</option><option>2:CaO</option><option>3:BaO</option>
<option>4:NaF</option><option>5:KF</option><option>6:KCl</option>
</select>
    <span style="margin-right: 200px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>Temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont.Temp:</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="2500" value="300" step="10"
style="width:360px" oninput="js.setContTemp()">
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>ball</option><option>bond length</option>
<option>bond direction</option><option>small ball + bond</option>
</select>
    <span style="margin-right: 80px;"></span>
<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>


</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# simulator run
exec_html_js()
print("-- start --")

# python control
for i in range(10):
  [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = ke + pe
  print(f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
  time.sleep(2)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# change theme, dispMode

import time

themeList = [ '0:NaCl', '1:MgO', '2:CaO','3:BaO', '4:NaF', '5:KF', '6:KCl' ]
dispModeList = [ 'ball', 'bond length', 'bond direction', 'small ball + bond' ]

# simulator run
exec_html_js()
print("-- start --")

# change theme and dispMode
for theme in [ 0, 4, 5, 6 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for dispMode in [ 0, 1, 2, 3 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print( "   -- dispMode:", dispModeList[dispMode], "--" )
    # get data and display
    [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData(0)' )
    energy = ke + pe
    print( f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
    time.sleep(3)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# system heat up and get data: timeList, temListp, energyList

import time

dispModeList = [ 'ball', 'bond length', 'bond direction' ]
timeList = []
tempList = []
energyList = []

# simulator run
exec_html_js()
print("-- start --")

# chage dispMode
dispMode = 1 # bond-length
eval_js( 'js.pysetDispMode({})'.format(dispMode) )
print( "-- dispMode:", dispModeList[dispMode], "--" )

# system heat up --> 2200K
for i in range(20):
  # set temperature
  temp = 300 + 100*i
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(2)

  # get data and display
  [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = ke + pe

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy)

  if i%2==0:
    print(f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)')

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# plot temp, energy

import matplotlib.pyplot as plt

# plot temp
plt.plot( timeList, tempList )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

# plot energy
plt.plot( timeList, energyList )
plt.xlabel('time (ps)')
plt.ylabel('total energy (J)')
plt.show()

In [ ]:
# colab AI wrote:
# prompt: in '# plot temp, energy' cell, plot one figure with dual y-axis

# plot temp, energy
import matplotlib.pyplot as plt

# plot temp and energy in one figure with dual y-axis
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('time (ps)')
ax1.set_ylabel('temp (K)', color=color)
ax1.plot(timeList, tempList, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('total energy (J)', color=color)  # we already handled the x-label with ax1
ax2.plot(timeList, energyList, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


In [ ]:
# get particles data and save

import time
import numpy as np

themeList = [ '0:NaCl', '1:MgO', '2:CaO','3:BaO', '4:NaF', '5:KF', '6:KCl' ]

# simulator run
exec_html_js()
print("-- start --")

# set theme
theme = 0 # '0:NaCl'
eval_js( 'js.pysetTheme({})'.format(theme) )
print( "-- theme:", themeList[theme], "--" )

time.sleep(5)

# get particle data and plot
print("-- get particles data --")
[ nowData, kindList, xxList, yyList, vxList, vyList ] = eval_js('js.pygetParticlesList()')
[sysTime, sysTemp, ke, pe ] = nowData
energy = ke + pe
print( f'got time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
print("-- get kindList, xxList, yyList, vxList, vyList,  length of list = ", len(vxList))

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

# convert [ kindList, xxList, yyList, vxList, vyList ] to np_data
np_data = np.array([ kindList, xxList, yyList, vxList, vyList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js023_data.npy'" )
np.save( 'js023_data.npy', np_data )

# save np_data.T as csv file
#np.savetxt('js023_data.csv', np_data.T, delimiter=',')

In [ ]:
# load particles data

import numpy as np

ionList =     [ "B",   "O",  "F",  "Na", "Mg", "Al", "Si", "Cl", "K", "Ca", "Ti", "Sr", "Ba" ]
SizeOf = np.array([ 0.23, 1.40, 1.33, 1.02, 0.72, 0.53, 0.40, 1.81, 1.38, 1.00, 0.61, 1.16, 1.36 ]) # bond radius (x 1e-10 m)
ColorOf = np.array([
    [0xff, 0x40, 0x60], [0x40, 0x00, 0xff], [0x00, 0x40, 0xff], [0xff, 0x00, 0x00],  # B, O, F, Na
    [0xff, 0x60, 0x00], [0xff, 0x40, 0x80], [0xff, 0x80, 0x40], [0x00, 0x00, 0xff],  # Mg, Al, Si, Cl
    [0xff, 0x40, 0x00], [0xff, 0x80, 0x00], [0xff, 0x80, 0x60], [0xff, 0xaf, 0x00], [0xff, 0xbf, 0x00]  # K, Ca, Ti, Sr, Ba
]) / 255
x_box_size, y_box_size = 5.0, 5.0

# load particles data
loaded_data = np.load('js023_data.npy')
print(f"-- load particles data  shape:{loaded_data.shape} ")
# set numpy array
Kind = loaded_data[0].astype(int)  # kind of every atom
X = loaded_data[1] * 1e9           # (nm) x-position of every atom
Y = loaded_data[2] * 1e9           # (nm) y-position of every atom
U = loaded_data[3]                 # (m/s) x-velocity of every atom
V = loaded_data[4]                 # (m/s) y-velocity of every atom
print(f'shape of X={X.shape}, Y={Y.shape}, U={U.shape}, V={V.shape}')
Vabs = np.sqrt(U**2 + V**2)        # (m/s) speed of every atom
C = ColorOf[Kind]                  # color of every atom
S = SizeOf[Kind] * 2.0              # size of every atom

In [ ]:
# plot particle position

import numpy as np
import matplotlib.pyplot as plt

# scatter plot
fig = plt.figure(figsize=(6, 6))
plt.scatter(X, Y, c=C, s=S*100)
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.xlabel('x ( nm )')
plt.ylabel('y ( nm )')
plt.title('NaCl crystal 2D')
plt.show()

In [ ]:
# velocity space

import numpy as np
import matplotlib.pyplot as plt

# scatter plot
vmax = 800
fig = plt.figure(figsize=(6, 6))
plt.scatter(U, V, c=C )
plt.xlabel('vx ( m/s )')
plt.ylabel('vy ( m/s )')
plt.title('velocity space')
plt.xlim(-vmax, vmax)
plt.ylim(-vmax, vmax)
plt.show()

In [ ]:
# hist Vabs

import numpy as np
import matplotlib.pyplot as plt

# get Na(kind==3), Cl(kind==7) data
Na_Vabs = Vabs[Kind==3]
Cl_Vabs = Vabs[Kind==7]

# plot
fig = plt.figure(figsize=(6, 6))
plt.hist(Na_Vabs, bins=30, color='red', alpha=0.5, label='Na')
plt.hist(Cl_Vabs, bins=30, color='blue', alpha=0.5, label='Cl')
plt.xlabel('speed (m/s)')
plt.ylabel('counts')
plt.legend()
plt.show()

In [ ]:
# bond (Na-Cl) length distribution

import numpy as np
import matplotlib.pyplot as plt

# normal bond length of Na - Cl
normal_bond_length = (SizeOf[3] + SizeOf[7]) * 0.1

# bond length distribution
bond_length_list = []
for i in range(len(X)):
  for j in range(i+1, len(X)):
    dx = X[i] - X[j]
    dy = Y[i] - Y[j]
    dr = np.sqrt(dx**2 + dy**2)
    if (dr < normal_bond_length * 1.2) and (Kind[i] != Kind[j]):
      bond_length_list.append(dr)

# plot bond length distribution
plt.hist(bond_length_list, bins=50)
plt.xlabel('bond length (nm)')
plt.ylabel('frequency')
plt.show()

In [ ]:
# data analysis with pandas

import pandas as pd
import numpy as np

# calculate correlation matrix
corr = np.corrcoef(np.array([X, Y, U, V]))
print(corr)

# print 1/sqrt(N)
print("1/sqrt(N) = ",1 / np.sqrt(len(X)))

# create a scatter matrix
pd.plotting.scatter_matrix(pd.DataFrame({'X': X, 'Y': Y, 'U': U, 'V': V}))